# 데이터 로드 및 확인

In [1]:
import json
import re

# 전처리 추가 (시스템 태그 삭제)
def clean_text(text):
    # 시스템 태그 제거
    text = re.sub(r"#@[^#]+#", "", text)
    # 중복된 기호, 공백 정리
    text = re.sub(r"[!@#\$%^&*\(\)\[\]_+=<>?/|\\~`\"';:]{2,}", "", text)
    text = re.sub(r"\s+", " ", text)
    return text.strip()


def preprocess_dialogues(json_path, save_path=None):
    import pandas as pd

    with open(json_path, 'r', encoding='utf-8') as f:
        data = json.load(f)

    pairs = []

    for dialogue in data['data']:
        body = dialogue['body']
        body.sort(key=lambda x: (x['turnID'], x['utteranceID']))
        
        prev_participant = None
        prev_text = ""
        
        for utt in body:
            pid = utt['participantID']
            text = clean_text(utt['utterance'])
            
            if not text:
                continue
            
            if prev_participant and pid != prev_participant:
                # 서로 다른 참여자 간 대화
                pairs.append((prev_text, text))
            
            prev_participant = pid
            prev_text = text
        

    if save_path:
        df = pd.DataFrame(pairs, columns=["input", "response"])
        df.to_csv(save_path, index=False, encoding='utf-8-sig')

    return pairs

In [2]:
# 전처리 실행
train_pairs = preprocess_dialogues("../data/text_dataset/한국어SNS_train/[라벨]한국어SNS_train/개인및관계.json", save_path="../data/text_dataset/save_path/train_pairs.csv")
valid_pairs = preprocess_dialogues("../data/text_dataset/한국어SNS_valid/[라벨]한국어SNS_valid/개인및관계.json", save_path="../data/text_dataset/save_path/valid_pairs.csv")

In [13]:
# 학습 샘플 확인
for i in range(20):
    print(f"Q: {train_pairs[i][0]}")
    print(f"A: {train_pairs[i][1]}")
    print("-" * 30)

Q: 잉ㅜㅜ
A: 돈따스
------------------------------
Q: 돈따스
A: 안보내줫어?
------------------------------
Q: 이거
A: 하 ......ㅡ
------------------------------
Q: 퀵으로한대서 두시까지오래 ㅋㅋㅋㅋ
A: ㅎㅎㅎㅎ오좋겠네
------------------------------
Q: ㅎㅎㅎㅎ오좋겠네
A: 잘잣어ㅋㅋㅋㅋㅋ
------------------------------
Q: 잘잣어ㅋㅋㅋㅋㅋ
A: ㅋㄱㅋㄱㄱㄱㄱ아니
------------------------------
Q: 머거
A: 잉
------------------------------
Q: 내돈가쓰...
A: ㅋㄱㄱㄱㄱㄱ맛있어
------------------------------
Q: 고로케도존맛탱
A: 사진찍엇어....
------------------------------
Q: 사진찍엇어....
A: ㅋㄱㄱㄱㅋ
------------------------------
Q: 학생이면좋구!
A: 훔
------------------------------
Q: 없는데...주변에...
A: 왜혼자다니냐고오.....
------------------------------
Q: 왜혼자다니냐고오.....
A: 아니
------------------------------
Q: 어케 친구가있냐..
A: 와 내친군학교나감
------------------------------
Q: 막졸업한애두굳
A: 없다구...
------------------------------
Q: 너무화난당..
A: 흠
------------------------------
Q: 흠
A: 근데오빠는말을또 잘해서 내가화내다보면결국내잘못
------------------------------
Q: 답답해진짱ㅋㅋ
A: 그럴때 억울하지 짖짜
------------------------------
Q: 오빠도 오늘 회식이야?
A: 아니
----------

### train_pairs.csv , valid_pairs.csv -> train.txt로 병합

In [8]:
import pandas as pd

def merge_csv_to_text(train_csv, valid_csv, output_txt):
    df_train = pd.read_csv(train_csv)
    df_valid = pd.read_csv(valid_csv)
    
    with open(output_txt, 'w', encoding='utf-8') as f:
        for df in [df_train, df_valid]:
            for i in range(len(df)):
                input_text = str(df.loc[i, "input"]).strip()
                response_text = str(df.loc[i, "response"]).strip()
                if input_text and response_text:
                    f.write(input_text + '\n')
                    f.write(response_text + '\n')
                    
merge_csv_to_text(
    "../data/text_dataset/save_path/train_pairs.csv",
    "../data/text_dataset/save_path/valid_pairs.csv",
    "../data/text_dataset/text_for_txt/train.txt"
)

----

# SentencePiece 토크나이저 학습
+ 띄어쓰기/어절 기반이 아닌 서브워드 단위로 토큰 분할
+ 유연하게 희귀 단어 처리 가능
+ 한국어 SNS 데이터에 잘 맞음

In [9]:
import os
import sentencepiece as spm

def train_sentencepiece(input_file, model_dir="../model/llm_model", model_name="chatbot_spm", vocab_size=16000):
    """
    SentencePiece 모델을 학습하고 지정한 위치에 저장합니다.

    :param input_file: 학습에 사용할 텍스트 파일 경로 (ex: train.txt)
    :param model_dir: 모델과 vocab 파일이 저장될 폴더 경로
    :param model_name: 저장될 모델 파일 이름 접두어
    :param vocab_size: 사용할 vocab 사이즈 (기본: 8000)
    """

    # 저장 경로 포함한 전체 prefix
    model_prefix = os.path.join(model_dir, model_name)

    # SentencePiece 학습 실행
    spm.SentencePieceTrainer.Train(
        f"--input={input_file} --model_prefix={model_prefix} --vocab_size={vocab_size} "
        "--model_type=bpe --character_coverage=1.0 --pad_id=0 --unk_id=1 --bos_id=2 --eos_id=3"
    )

    print(f"✅ 모델 저장 완료: {model_prefix}.model")
    print(f"✅ 단어 사전 저장 완료: {model_prefix}.vocab")

In [10]:
# 실행 예시
train_sentencepiece("../data/text_dataset/text_for_txt/train.txt", model_dir="../model/llm_model", model_name="chatbot_spm")

✅ 모델 저장 완료: ../model/llm_model/chatbot_spm.model
✅ 단어 사전 저장 완료: ../model/llm_model/chatbot_spm.vocab


sentencepiece_trainer.cc(177) LOG(INFO) Running command: --input=../data/text_dataset/text_for_txt/train.txt --model_prefix=../model/llm_model/chatbot_spm --vocab_size=16000 --model_type=bpe --character_coverage=1.0 --pad_id=0 --unk_id=1 --bos_id=2 --eos_id=3
sentencepiece_trainer.cc(77) LOG(INFO) Starts training with : 
trainer_spec {
  input: ../data/text_dataset/text_for_txt/train.txt
  input_format: 
  model_prefix: ../model/llm_model/chatbot_spm
  model_type: BPE
  vocab_size: 16000
  self_test_sample_size: 0
  character_coverage: 1
  input_sentence_size: 0
  shuffle_input_sentence: 1
  seed_sentencepiece_size: 1000000
  shrinking_factor: 0.75
  max_sentence_length: 4192
  num_threads: 16
  num_sub_iterations: 2
  max_sentencepiece_length: 16
  split_by_unicode_script: 1
  split_by_number: 1
  split_by_whitespace: 1
  split_digits: 0
  pretokenization_delimiter: 
  treat_whitespace_as_suffix: 0
  allow_whitespace_only_pieces: 0
  required_chars: 
  byte_fallback: 0
  vocabulary_ou

bpe_model_trainer.cc(268) LOG(INFO) Added: freq=6426 size=1060 all=870633 active=58186 piece=거나
bpe_model_trainer.cc(268) LOG(INFO) Added: freq=6274 size=1080 all=877609 active=65162 piece=시반
bpe_model_trainer.cc(268) LOG(INFO) Added: freq=6097 size=1100 all=884751 active=72304 piece=▁줘
bpe_model_trainer.cc(159) LOG(INFO) Updating active symbols. max_freq=6097 min_freq=93
bpe_model_trainer.cc(268) LOG(INFO) Added: freq=5978 size=1120 all=890247 active=49599 piece=겟어
bpe_model_trainer.cc(268) LOG(INFO) Added: freq=5818 size=1140 all=895083 active=54435 piece=▁어디서
bpe_model_trainer.cc(268) LOG(INFO) Added: freq=5694 size=1160 all=901343 active=60695 piece=▁뀨
bpe_model_trainer.cc(268) LOG(INFO) Added: freq=5586 size=1180 all=906583 active=65935 piece=리면
bpe_model_trainer.cc(268) LOG(INFO) Added: freq=5464 size=1200 all=914078 active=73430 piece=▁끊
bpe_model_trainer.cc(159) LOG(INFO) Updating active symbols. max_freq=5464 min_freq=90
bpe_model_trainer.cc(268) LOG(INFO) Added: freq=5379 siz

## Tokenizer 클래스 기능
1. 텍스트 -> 토큰 ID (정수 시퀀스) 변환 (encode)
2. 토큰 ID -> 텍스트 복원 (decode)
3. special token (pad, bos, eos 등) 관리 

In [16]:
class Tokenizer:
    def __init__(self, model_path: str):
        self.sp = spm.SentencePieceProcessor()
        self.sp.load(model_path)
        
        self.pad_id = self.sp.pad_id()
        self.unk_id = self.sp.unk_id()
        self.bos_id = self.sp.bos_id()
        self.eos_id = self.sp.eos_id()
    
    def encode(self, text: str, add_bos=True, add_eos=True) -> list:
        tokens = self.sp.encode(text, out_type=int)
        if add_bos:
            tokens = [self.bos_id] + tokens
        if add_eos:
            tokens = tokens + [self.eos_id]
        return tokens
    
    def decode(self, ids: list) -> str:
        ids = [i for i in ids if i not in [self.bos_id, self.eos_id, self.pad_id]]
        return self.sp.decode(ids)
    
    def vacab_size(self):
        return self.sp.get_piece_size()

In [17]:
# 테스트
tokenizer = Tokenizer("../model/llm_model/chatbot_spm.model")

text = "오늘 날씨 어때?"
encoded = tokenizer.encode(text)
decoded = tokenizer.decode(encoded)

print("✅ 원문:", text)
print("🧠 인코딩:", encoded)
print("🔁 디코딩:", decoded)

✅ 원문: 오늘 날씨 어때?
🧠 인코딩: [2, 57, 1602, 1060, 8898, 3]
🔁 디코딩: 오늘 날씨 어때?
